In [1]:
import time
start_time = time.time()
import os
import numpy as np
from keras import optimizers
from keras.preprocessing import image
from keras.layers import Dense, Conv2D, MaxPooling2D, BatchNormalization, GlobalAveragePooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.optimizers import adam
from keras import models
import cv2
import dlib
import pandas as pd
import matplotlib.pyplot as plt
import sys
import time 
import progressbar
from tqdm.notebook import tqdm_notebook

Using TensorFlow backend.


In [2]:
from sklearn import svm, datasets
from sklearn.dummy import DummyClassifier
from sklearn.svm import SVC
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split, GridSearchCV, learning_curve, ShuffleSplit
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix, recall_score, precision_score, f1_score

In [3]:
from tensorflow.python.keras.applications.vgg16 import preprocess_input
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.keras.callbacks import EarlyStopping, ModelCheckpoint

In [5]:
df = pd.read_csv('../../dataset/cartoon_set/labels.csv', sep = '\t')
df = df.drop(columns = [df.columns[0]]).drop(columns = [df.columns[2]])
df['eye_color'] = df['eye_color'].apply(str)
df

,eye_color,file_name
0,1,0.png
1,2,1.png
2,2,2.png
3,2,3.png
4,0,4.png
...,...,...
9995,3,9995.png
9996,0,9996.png
9997,1,9997.png
9998,0,9998.png


In [13]:
img = ('../../dataset/cartoon_set/img')
# training, testing = train_test_split(df, random_state=0)
for x in [0.6, 0.65, 0.7, 0.75, 0.8]:
    for y in [0.8, 0.8, 0.85, 0.85, 0.9]:
        training, validating, testing = np.split(df.sample(frac=1), [int(.6*len(df)), int(.8*len(df))]) #splitting at n-array
# print(training, validating, testing)

In [14]:
# set up data generator
data_generator = ImageDataGenerator(
    rescale = 1./255.,
#     validation_split = 0.25,
    horizontal_flip=True,
    vertical_flip=True   
)

# # get batches of training images from the df
# train_generator = data_generator.flow_from_dataframe(
#         dataframe=train_set,
#         x_col="file_name",
#         y_col="face_shape",
#         target_size=(178, 218),
#         batch_size=12,
#         class_mode='categorical')

# validate_generator = data_generator.flow_from_dataframe(
#         dataframe=train_set,
#         x_col="file_name",
#         y_col="face_shape",
#         target_size=(178, 218),
#         batch_size=12,
#         class_mode='categorical')

# Get batches of training dataset from the dataframe
print("Training Dataset Preparation: ")
train_generator = data_generator.flow_from_dataframe(
        dataframe = training, directory = img ,
        x_col = "file_name", y_col = "eye_color",
        class_mode = 'categorical', target_size = (30,30),
        batch_size = 32, subset = 'training') 
   
# Get batches of validation dataset from the dataframe
print("\nValidation Dataset Preparation: ")
validation_generator = data_generator.flow_from_dataframe(
        dataframe = validating, directory = img ,
        x_col = "file_name", y_col = "eye_color",
        class_mode = 'categorical', target_size = (30,30),
        batch_size = 32, subset = 'training')

Training Dataset Preparation: 
Found 6000 validated image filenames belonging to 5 classes.

Validation Dataset Preparation: 
Found 2000 validated image filenames belonging to 5 classes.


In [15]:
# starting point 
my_model= models.Sequential()

# Add first convolutional block
my_model.add(Conv2D(16, (3, 3), activation='relu', padding='same', 
                    input_shape=(30,30,3))) #height, width, depth
my_model.add(MaxPooling2D((2, 2), padding='same'))
# second block
my_model.add(Conv2D(32, (3, 3), activation='relu', padding='same')) #Convolution: filters, kernel_size that specifies the height and width of the 2D convolution window, p padding layers so dimensions of input = output
my_model.add(MaxPooling2D((2, 2), padding='same'))
# third block
my_model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
my_model.add(MaxPooling2D((2, 2), padding='same'))
# fourth block
my_model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
my_model.add(MaxPooling2D((2, 2), padding='same'))
# global average pooling
#my_model.add(GlobalAveragePooling2D())
# fully connected layer
#my_model.add(Dense(64, activation='relu'))
#my_model.add(BatchNormalization())
# make predictions
my_model.add(Flatten())
my_model.add(Dense(5, activation='softmax'))
# Show a summary of the model. Check the number of trainable parameters
my_model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 30, 30, 16)        448       
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 15, 15, 16)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 15, 15, 32)        4640      
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 8, 8, 32)          0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 8, 8, 64)          18496     
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 4, 4, 64)          0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 4, 4, 128)        

In [16]:
# # use early stopping to optimally terminate training through callbacks

# es=EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5)

# # save best model automatically
# mc= ModelCheckpoint('/../../dataset/cartoon_set', monitor='val_loss', 
#                     mode='min', verbose=1, save_best_only=True)
# cb_list=[es,mc]


# compile model 
my_model.compile(optimizer='adam', loss='categorical_crossentropy', 
                 metrics=['accuracy'])

In [ ]:
batch_size = 32
history = my_model.fit_generator(
                                train_generator,
                                epochs=25,
                                steps_per_epoch=train_generator.samples // batch_size,
                                validation_data=validation_generator,
                                validation_steps=validation_generator.samples // batch_size
                                )

Epoch 1/25
187/187 [==============================] - 89s 476ms/step - loss: 1.6116 - accuracy: 0.1976 - val_loss: 1.6132 - val_accuracy: 0.2006
Epoch 2/25
187/187 [==============================] - 68s 366ms/step - loss: 1.6099 - accuracy: 0.2016 - val_loss: 1.6092 - val_accuracy: 0.2114
Epoch 3/25
187/187 [==============================] - 67s 358ms/step - loss: 1.6098 - accuracy: 0.1952 - val_loss: 1.6093 - val_accuracy: 0.1951
Epoch 4/25
187/187 [==============================] - 66s 355ms/step - loss: 1.6099 - accuracy: 0.2036 - val_loss: 1.6087 - val_accuracy: 0.1956
Epoch 5/25
187/187 [==============================] - 67s 361ms/step - loss: 1.6096 - accuracy: 0.2012 - val_loss: 1.6100 - val_accuracy: 0.1992
Epoch 6/25
187/187 [==============================] - 67s 357ms/step - loss: 1.6096 - accuracy: 0.1994 - val_loss: 1.6046 - val_accuracy: 0.1992
Epoch 7/25
187/187 [==============================] - 65s 346ms/step - loss: 1.5767 - accuracy: 0.2394 - val_loss: 1.4261 - val_ac

In [ ]:
print("--- %s seconds ---" % (time.time() - start_time))